### 환경설정

In [ ]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git 
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets

참조

https://github.com/Beomi/KoAlpaca

https://colab.research.google.com/gist/Beomi/f163a6c04a869d18ee1a025b6d33e6d8/2023_05_26_bnb_4bit_koalpaca_v1_1a_on_polyglot_ko_12_8b.ipynb

https://wikidocs.net/238524


---

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset

In [ ]:
print("CUDA 사용 가능:", torch.cuda.is_available())
print("GPU 이름:", torch.cuda.get_device_name(0))

print("Torch CUDA 지원 여부:", torch.cuda.is_available())
print("CUDA 버전:", torch.version.cuda)
print("PyTorch 버전:", torch.__version__)

---

### 모델, 토크나이저 로드 및 LoRA 설정, DataLoad

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=False)

tokenizer.pad_token = "<|finetune_right_pad_id|>"
tokenizer.pad_token_id = 128004
tokenizer.padding_side = 'right'

print(model)

샘플이 너무 적으면 과적합(Overfitting) 위험이 크므로 데이터를 증강(Augmentation)하는 것이 필요.

In [ ]:
from datasets import load_dataset

# LLaMA 3 Chat 템플릿 적용
chat_template = """<|begin_of_text|><|start_header_id|>지시사항<|end_header_id|>
{SYSTEM}<|eot_id|><|start_header_id|>입력<|end_header_id|>
{INPUT}<|eot_id|><|start_header_id|>응답<|end_header_id|>
{OUTPUT}<|eot_id|>"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    
    for instruction, output in zip(instructions, outputs):
        text = chat_template.format(
            SYSTEM="아래는 작업을 설명하는 지시사항입니다. 입력된 내용을 바탕으로 적절한 응답을 작성하세요.",
            INPUT= instruction,
            OUTPUT= output 
        ) 
        
        texts.append(text)
    
    return {"text": texts}

# 데이터셋 로드
dataset = load_dataset("json", data_files="../00_Data/KoAlpaca_train.json")
# 데이터셋 변환
dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=['instruction', 'output'])

split_data = dataset['train'].train_test_split(test_size=0.05, seed=42)
train_data, val_data = split_data['train'], split_data['test']


In [ ]:
df = train_data.to_pandas()  # "train" 데이터셋을 pandas로 변환
print(df.head())  # 첫 5개 샘플 확인

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()#훈련 시 메모리 절약 (출력값을 필요할 때만 계산)
model = prepare_model_for_kbit_training(model)

In [ ]:
#LoRA(PEFT) 설정을 적용하여 기존 모델을 효율적으로 미세 조정
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",# LoRA를 적용할 작업 유형 (CAUSAL_LM: 언어 모델)
    r=8,# LoRA 랭크 (적은 수록 가벼움, 크면 성능 향상 가능)
    lora_alpha=16,   # 일반적으로 LoRA의 효과를 조절하는 파라미터 (값이 크면 LoRA 가중치의 영향 증가)
    lora_dropout=0.05, # Dropout 확률 (일반적으로 0~0.1 추천)
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
)

---

### Finetuning

In [ ]:
import transformers
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer

train_args = TrainingArguments(
    per_device_train_batch_size=2, # 배치 크기 (GPU당 샘플 개수)
    gradient_accumulation_steps=1,  # 메모리 최적화 Gradient Accumulation 누적 스텝 (메모리 부족 시 증가 가능)
    gradient_checkpointing=True, # 활성화하면 GPU 메모리 사용 감소 가능
    num_train_epochs=1, # 전체 데이터셋을 몇 번 반복해서 학습할 것인지
    # warmup_steps=30,  # 학습률을 서서히 증가시키는 단계 (0~100)
    # max_steps=300 ,  # 최대 학습 스텝
    learning_rate=2e-4,  # 학습률 (기본 2e-4)
    lr_scheduler_type="linear", # 학습률 스케줄러 종류 ( linear, cosine, constant )
    weight_decay=0.01,
    bf16=True, # 정밀도 ( FP32, FP16, BF16 )
    warmup_ratio=0.1, # 전체 Step 수의 10% 동안 학습률을 점진적으로 증가
    optim="adamw_torch", # paged_adamw_8bit (VRAM절약 성능 하락) adamw_torch(정확도 높음 메모리사용량 높음)
    seed=42,
    output_dir="../01_Models/01_RoLaModels",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="epoch", # 에포크 단위로 저장
    report_to="none", # WandB, TensorBoard 등 로그 저장 안 함 (필요시 변경)
    # log_level="debug",
)

# Trainer Setup
trainer = SFTTrainer(
    model=model,
    peft_config=lora_config,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=train_args,

    # dataset_text_field="text",
    # max_seq_length=1024,
    # packing=False,
)

model.config.use_cache = False
# 학습 시작
trainer.train()


In [ ]:
model.eval() # 모델의 가중치는 변경하지 않고, forward 연산만 수행함.
model.config.use_cache = True  # 이전 계산 결과를 저장하고 사용	추론 속도 빨라짐, 메모리 사용 증가

---

### 테스트

In [ ]:
# 채팅 스타일 프롬프트 (Llama-3의 Chat 모델용)
chat_prompt = """<|begin_of_text|><|start_header_id|>지시사항<|end_header_id|>
아래는 작업을 설명하는 지시사항입니다. 입력된 내용을 바탕으로 적절한 응답을 작성하세요.<|eot_id|>
<|start_header_id|>입력<|end_header_id|>
다시 합창 합시다' 처럼 거꾸로 읽어도 같은 문장이 영어에도 있나요? 또한 다른 나라의 언어에도 있는 건가요?<|eot_id|>
<|start_header_id|>응답<|end_header_id|>
"""
# 토큰화 및 모델 실행
input_ids = tokenizer(chat_prompt, return_tensors="pt").input_ids.to("cuda")
with torch.no_grad():
    output_ids = model.generate(input_ids, max_new_tokens=100, temperature=0.7, top_p=0.9, do_sample=True)

# 출력 변환
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("LLM 응답:", output_text)


---

###  기존 모델과 로라 병합 (어뎁터 유지) 풀파인튜닝 저장 

merge_and_unload 양자화 상태로 해버리면 로라 어뎁터가 망가져버림 그래서 양자화 하지 않고 저장해줘야함

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-3.1-8B-Instruct"
#로라 모델경로를 확인해봐야 해요
peft_model_id = "../01_Models/01_RoLaModels/checkpoint-19" 

loadModel = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # float16로 유지 bfloat16
    device_map="auto"
)

loadModel = PeftModel.from_pretrained(loadModel, peft_model_id, device_map="auto")
loadtokenizer = AutoTokenizer.from_pretrained(model_id)


In [ ]:
loadModel = loadModel.merge_and_unload() #실제 병합
merged_model_path = "../01_Models/02_FullFinetuningModels"
loadModel.save_pretrained(merged_model_path)
loadtokenizer.save_pretrained(merged_model_path)

---

### GGUF  llama cpp 사용

참조

https://m.blog.naver.com/112fkdldjs/223513042256

https://github.com/ollama/ollama/issues/4442

https://github.com/ollama/ollama/issues/4572

https://github.com/teddylee777/langserve_ollama/tree/main/ollama-modelfile/Llama-3-8B-Instruct

Q8_0: 8비트 양자화 모델로, 원본 모델의 품질을 거의 그대로 유지하면서 크기를 절반으로 줄입니다.

Q6_K, Q5_K_M, Q5_K_S: 6비트와 5비트 양자화 모델들로, 품질 손실은 미미하지만 크기가 더 작습니다.

Q4_K_M, Q4_K_S: 4비트 양자화 모델로, 약간의 품질 손실이 있지만 크기가 매우 작습니다.

In [ ]:
!python ../llama.cpp/convert_hf_to_gguf.py ../01_Models/02_FullFinetuningModels --outtype q8_0